# Using Pixi for reproducable environments

OpenMDAO requires a number of dependencies to fully take advantage of its capabilities. Sometimes finding versions of dependencies which are compatible and are known to function correctly is difficult.

OpenMDAO uses [Pixi](https://pixi.sh) for dependency management. Pixi is especially useful because some OpenMDAO dependencies (like MPI, PETSc) are not available on PyPI but are needed for parallel computing features.

Using Pixi allows users and developers to get consistent, reproducable environments.

## Installing Pixi

Before users can utilize Pixi, it needs to be installed:

```bash
curl -fsSL https://pixi.sh/install.sh | bash
```

## Getting the Pixi environments

Pixi stores information about OpenMDAO environments in a _manifest_ file (`pixi.toml`) and a _lock_ file (`pixi.lock`) in the root directory of the OpenMDAO git repository.

The intent is that the lock file will be tested in our CI machinery, and should provide a consistent environment for as many of our users as possible.

Users who want to develop OpenMDAO itself can setup the Pixi environments from the cloned repository.

### Option 1: Using the environment from the OpenMDAO git repo
```bash
# Clone the repository
git clone https://github.com/OpenMDAO/OpenMDAO.git

# Enter the directory with the pixi.toml and pixi.lock files
cd OpenMDAO
```

### Option 2: Downloading the Pixi manifest and lock files
```bash
# Download the latest Pixi configuration...
curl -O https://raw.githubusercontent.com/OpenMDAO/OpenMDAO/master/pixi.toml
curl -O https://raw.githubusercontent.com/OpenMDAO/OpenMDAO/master/pixi.lock

# Alternatively, for a specific release...
curl -O https://raw.githubusercontent.com/OpenMDAO/OpenMDAO/3.42.0/pixi.toml
curl -O https://raw.githubusercontent.com/OpenMDAO/OpenMDAO/3.42.0/pixi.lock
```

Pixi files are only available as of OpenMDAO version 3.42.0.

### Installing the desired environment

From the directory with the manifest and lock files:

```bash
# Install the default environment (no mpi)
pixi install --frozen

# Or, install the environment with all dependencies
pixi install -e all --frozen
```

Using **--frozen** causes Pixi to use the exact versions for the environment as defined in the `pixi.lock` file. This is what we test against on CI and should be the most reliable environment.

You can remove `--frozen` and Pixi will attempt to install the latest possible dependencies for most environments.



### Using the given environment via shell

You can then either shell into the environment (more akin to activating a conda environment), or you can run a command using that environment.

```bash
# Shell into the all environment (if installed)...
pixi shell -e all

# Install OpenMDAO in developer mode (if in the OpenMDAO git repo)...
python -m pip install -e .

# OR, Install OpenMDAO from pypi...
python -m pip install openmdao
```

That last step may seem a bit odd, but OpenMDAO is not a dependency of itself. **You will need to install OpenMDAO into whatever pixi environment you are using.** You only need to do this once after a given environment is installed.

Now the environment is ready to be used

```bash
# Run a script in the environment that we have shelled into...
python my_openmdao_script.py
```

More information about our Pixi implementation, including available environments and other ways of using it, are avaiable in the developer section of our documentation.

### Using the given environment via `pixi run`

Alternatively, users can run commands given a desired Pixi environment without shelling into it.

As before, make sure OpenMDAO has been installed in the environment:

```bash
# Install OpenMDAO from pypi into the all environment
pixi run -e all python -m pip install openmdao
```

Or, install a local OpenMDAO git repository in developer mode into the all environment:
```bash
# Install OpenMDAO from a local git repository
cd OpenMDAO
pixi run python -m pip install -e .
```

And now, run an OpenMDAO python script using the given environment
```bash
pixi run python path/to/my_openmdao_script.py
```

## Available Environments

- default: General development environment with JAX, numba, visualization, notebooks, and testing support, but no MPI.
           Best for most users and everyday development work.
           Usage: pixi shell  (or pixi shell -e default)

- minimal: Default dependencies only plus testing related dependencies.
           Used for testing in the absence of dependencies.
           Usage: pixi shell -e minimal

- all: Comprehensive environment with all optional dependencies except documenation and testing-related ones.
       This will give most users the "full" OpenMDAO capability.
       Usage: pixi shell -e all

- dev: The "all" environment with the addition of documentation and packaging-related dependencies.
       Usage: pixi shell -e docs

- oldest: Includes oldest supported versions of all dependencies except packaging-related ones.
          Usage: pixi shell -e oldest

- latest: Includes latest versions of all dependencies except packaging-related ones.

In [ ]:
try:
    import openmdao.api as om  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om  # noqa: F401
